In [6]:
!uv pip install unsloth llmcompressor -qU --system

In [7]:
# Then run:
import transformers
print(transformers.__file__)
print(transformers.__version__)

/usr/local/lib/python3.11/dist-packages/transformers/__init__.py
4.55.2


In [9]:
# model_to_compress = "Trelis/arc-1-fake-ttt-blended-c802"
# output_dir = f"{model_to_compress}-fp8"

# import transformers
# from transformers import AutoTokenizer, AutoModelForCausalLM
# print(transformers.__version__)

# model = AutoModelForCausalLM.from_pretrained(model_to_compress, torch_dtype="auto", trust_remote_code=True, device_map="auto")
# tokenizer = AutoTokenizer.from_pretrained(model_to_compress, trust_remote_code=True)

In [1]:
import os
os.environ["HF_HOME"] = "/workspace"
os.environ["HF_HUB_CACHE"] = "/workspace/hub" # (recommended) override just the repo cache
print(os.environ["HF_HOME"])

import unsloth
from unsloth import FastLanguageModel

model_to_compress = "Trelis/arc-1-fake-ttt-blended-c802"
output_dir = f"{model_to_compress}-fp8"

model_max_length = 32768 # max eval set length is around 19,400 tokens. Add some headroom for responses and arc-agi-2 test lengths. This doesn't matter a whole lot because the rows of data are padded to max length.
lora_rank = 128

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_to_compress,
    max_seq_length = model_max_length,   # Context length - can be longer, but uses more memory
    load_in_4bit = False,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    # cache_dir = '/workspace',
    # token = "hf_...",      # use one if using gated models
    
    # for using fast_inference
    # fast_inference = False, # removing this as it takes up VRAM
    # gpu_memory_utilization=0.3, # not needed if not using vLLM for inference
)

/workspace
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: We'll be using `/tmp/unsloth_compiled_cache` for temporary Unsloth patches.
Standard import failed for UnslothBCOTrainer: No module named 'UnslothBCOTrainer'. Using tempfile instead!


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


==((====))==  Unsloth 2025.8.5: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    NVIDIA H200. Num GPUs = 1. Max memory: 139.719 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 9.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [2]:
from llmcompressor import oneshot
from llmcompressor.modifiers.quantization import QuantizationModifier

# Configure the simple PTQ quantization
recipe = QuantizationModifier(
  targets="Linear", scheme="FP8_DYNAMIC", ignore=["lm_head"])

# Apply the quantization algorithm.
oneshot(model=model, recipe=recipe)

# Save the model.
new_name = model_to_compress.rstrip("/").split("/")[-1] + "-FP8-Dynamic"
SAVE_DIR = 'compressed/' + new_name
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

2025-08-15T08:25:36.122118+0000 | reset | INFO - Compression lifecycle reset
2025-08-15T08:25:36.125013+0000 | from_modifiers | INFO - Creating recipe from modifiers
2025-08-15T08:25:36.196211+0000 | initialize | INFO - Compression lifecycle initialized for 1 modifiers
2025-08-15T08:25:36.196751+0000 | IndependentPipeline | INFO - Inferred `DataFreePipeline` for `QuantizationModifier`


Calibrating weights: 100%|██████████| 799/799 [00:00<00:00, 7622.57it/s]

2025-08-15T08:25:36.315506+0000 | finalize | INFO - Compression lifecycle finalized for 1 modifiers
2025-08-15T08:25:36.317466+0000 | post_process | WARNING - Optimized model is not saved. To save, please provide`output_dir` as input arg.Ex. `oneshot(..., output_dir=...)`
2025-08-15T08:25:36.335636+0000 | get_model_compressor | INFO - skip_sparsity_compression_stats set to True. Skipping sparsity compression statistic calculations. No sparsity compressor will be applied.



Compressing model: 547it [00:09, 56.69it/s]


('arc-1-fake-ttt-blended-c802-FP8-Dynamic/tokenizer_config.json',
 'arc-1-fake-ttt-blended-c802-FP8-Dynamic/special_tokens_map.json',
 'arc-1-fake-ttt-blended-c802-FP8-Dynamic/chat_template.jinja',
 'arc-1-fake-ttt-blended-c802-FP8-Dynamic/vocab.json',
 'arc-1-fake-ttt-blended-c802-FP8-Dynamic/merges.txt',
 'arc-1-fake-ttt-blended-c802-FP8-Dynamic/added_tokens.json',
 'arc-1-fake-ttt-blended-c802-FP8-Dynamic/tokenizer.json')

In [4]:
from huggingface_hub import HfApi

owner = "Trelis"  # use your exact org handle/case
repo_id = f"{owner}/{new_name}"

api = HfApi()
api.create_repo(repo_id=repo_id, repo_type="model", exist_ok=True)  # add private=True if desired

# # Ensure large weights go to LFS
# from pathlib import Path
# (Path(output_dir)/".gitattributes").write_text(
#     "*.safetensors filter=lfs diff=lfs merge=lfs -text\n*.bin filter=lfs diff=lfs merge=lfs -text\n"
# )

api.upload_folder(
    repo_id=repo_id,
    folder_path=SAVE_DIR,
    repo_type="model",
    commit_message="Add FP8 (W8A8 dynamic activations) compressed checkpoint",
)
print(f"Pushed to https://huggingface.co/{repo_id}")

  0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/4.41G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Pushed to https://huggingface.co/Trelis/arc-1-fake-ttt-blended-c802-FP8-Dynamic
